In [14]:
import pandas as pd 
import numpy as np
import sqlalchemy
import pyodbc

import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.window as w
import pyspark.sql.types as t

In [15]:
spark = SparkSession.builder.appName('TheVoice-ETL').getOrCreate()

In [16]:
# Load into dataframes
call_type_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\call_type.parquet')
countries_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\countries.parquet')
customer_mrr =          spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer.parquet')
customer_invoice_mrr =  spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer_invoice.parquet')
customer_lines_mrr =    spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer_lines.parquet')
opfileopp_mrr =         spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\pfileopp.parquet')
package_catalog_mrr =   spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\package_catalog.parquet')
usage_main_mrr =        spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\usage_main.parquet')
xxCountryType_mrr =     spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\xxCountryType.parquet')

In [17]:
# DimCallTypes staging

# variable set
price_per_minute = 0.5

# window for the key column
windowSpec_callType = w.Window.orderBy('call_type_code')

# final df 
DimCallTypes_stg = call_type_mrr.withColumn('KeyCallType', f.row_number().over(windowSpec_callType) + 1000)\
    .withColumnRenamed('call_type_code', 'DescCallTypeCode')\
    .withColumnRenamed('call_type_desc', 'DescCallType')\
    .withColumn('DescFullCallType', 
        f.concat_ws('-', f.col('DescCallTypeCode'), f.col('DescCallType')))\
    .withColumn('DescCallTypePriceCategory', f.when(f.col('priceperminuter') > price_per_minute, 'Discounted Price').otherwise('Normal Price'))\
    .withColumnRenamed('call_type', 'DescCallTypeCategory')\
    .select('KeyCallType', 'DescCallTypeCode', 'DescCallType', 'DescFullCallType', 'DescCallTypePriceCategory', 'DescCallTypeCategory')#.show()


In [18]:
# DimCountries staging

# select proper column and rename the country code column
xxCountryType_mrr = xxCountryType_mrr.select('COUNTRY_CODE', 'COUNTRY_PRE').withColumnRenamed('COUNTRY_CODE','COUNTRY_CODE2')

# final df
DimCountries_stg = countries_mrr.select('COUNTRY_CODE', 'REGION', 'AREA').join(
    xxCountryType_mrr,
    countries_mrr['COUNTRY_CODE'] == xxCountryType_mrr['COUNTRY_CODE2'],
    how='inner')\
    .withColumnRenamed('COUNTRY_PRE', 'KeyCountry')\
    .withColumnRenamed('COUNTRY_CODE', 'DescCountry')\
    .withColumnRenamed('REGION', 'DescRegion')\
    .withColumnRenamed('AREA', 'DescArea')\
    .select('KeyCountry', 'DescCountry', 'DescRegion', 'DescArea')#.show(5)

In [19]:
# DimPackageCatalog staging
DimPackageCatalog_stg = package_catalog_mrr.withColumnRenamed('PACKAGE_NUM','KeyPackage')\
.withColumnRenamed('pack_desc', 'DescPackage')\
.withColumnRenamed('createdate', 'DatePackageCreation')\
.withColumnRenamed('enddate', 'DatePackageEnd')\
.withColumn('DescPackageStatus',
    f.when(f.col('status') == 1, 'Active').otherwise('Inactive'))\
.withColumn('CodePackageActivitiesDays', f.date_diff(f.col('DatePackageEnd'), f.col('DatePackageCreation')))

In [20]:
opfileopp_mrr.show(n=5)

+-----+----------+------+
|OPCCC|     OPDDD|prepre|
+-----+----------+------+
|   50|    פלאפון|    50|
|   52|     סלקום|    52|
|   53|הוט מובייל|    53|
|   54|     אורנג|    54|
|   58|גולן טלקום|    58|
+-----+----------+------+



In [21]:
# DimOperators staging
DimOperators_stg = opfileopp_mrr.withColumnRenamed('OPCCC', 'KeyOperator')\
    .withColumn('DescOperator', f.concat_ws('-', f.col('prepre').cast('string'), f.col('OPDDD').cast('string')))\
    .withColumnRenamed('prepre', 'DescKeyPrefix')\
    .select('KeyOperator', 'DescOperator', 'DescKeyPrefix')

In [22]:
customer_mrr.show(5)

customer_lines_mrr.show(5)

opfileopp_mrr.show(5)

+-----------+------------+-----------------+-------------------+--------------------+-----------+
|customer_id| CUST_NUMBER|        cust_name|            address|         insert_date|update_date|
+-----------+------------+-----------------+-------------------+--------------------+-----------+
|          1|972541185146|     Eugene Huang|         2243 W St.|2014-01-15 16:25:...|       NULL|
|          2|972547493463|     Ruben Torres|   5844 Linden Land|2014-01-15 16:25:...|       NULL|
|          3|972549738380|      Christy Zhu|   1825 Village Pl.|2014-01-15 16:25:...|       NULL|
|          4|972540643054|Elizabeth Johnson|7553 Harness Circle|2014-01-15 16:25:...|       NULL|
|          5|972546595053|       Julio Ruiz|7305 Humphrey Drive|2014-01-15 16:25:...|       NULL|
+-----------+------------+-----------------+-------------------+--------------------+-----------+
only showing top 5 rows

+-----------+--------------------+-------+------+--------+----------------+------------------

"extract the operator code from the phone number and  bring the operator name from opfileopp. 
The assumption is that a phone number (without country and opperator prefix) has 7 digits and that country code may have 1 or 2 or 3 digits. 
For example,
-  for phone_no +972549972307, the opertator will be the operator name for prefix 54.
- for phone_no, +12127114685, the operator will be the name for prefix 212
if operator name not found then put 'Unknown'"


In [33]:
# DimCustomers staging
customer_mrr = spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer.parquet')
customer_mrr.join(customer_lines_mrr.select('PHONE_NO'), 
        customer_mrr['CUST_NUMBER'] == customer_lines_mrr['PHONE_NO'], how='inner')\
    .withColumnRenamed('customer_id', 'KeyCustomer')\
    .withColumn( 'CustomerLineOperatorPrefixForJoin',
        f.when(f.length(f.col('PHONE_NO').cast('string')) == 12, 
            f.substr(f.col('PHONE_NO').cast('string'), f.lit(4), f.lit(2)))\
            .otherwise(f.substr(f.col('PHONE_NO').cast('string'), f.lit(2), f.lit(3))))\
    .withColumn('CustomercountryPrefixForJoin',
        f.when(f.length(f.col('PHONE_NO').cast('string')) == 12, 
            f.substr(f.col('PHONE_NO').cast('string'), f.lit(1), f.lit(3)))\
            .otherwise(f.substr(f.col('PHONE_NO').cast('string'), f.lit(1), f.lit(1))))\
    .withColumnRenamed('cust_name', 'DescCustomerName')\
    .withColumnRenamed('address', 'DescCustomerAddress')\
    .join(opfileopp_mrr.select('OPCCC', 'OPDDD'),#
        f.col('CustomerLineOperatorPrefixForJoin').cast('int') == f.col('OPCCC').cast('int'), how='left')\
    .withColumn('operatorName',
         f.when(f.isnull(f.col('OPDDD')), 'Unknown').otherwise(f.col('OPDDD'))
    )
    .select('KeyCustomer', 'PHONE_NO', 'CustomerLineOperatorPrefixForJoin', 'CustomerCountryPrefixForJoin', 'DescCustomerName', 'DescCustomerAddress', f.col('OPDDD').alias('DescCustomerLineOperator'))\
    .show(5)#.orderBy(f.col('CustomerLineOperatorPrefixForJoin').asc())

+-----------+------------+---------------------------------+----------------------------+-----------------+-------------------+------------+
|KeyCustomer|    PHONE_NO|CustomerLineOperatorPrefixForJoin|CustomerCountryPrefixForJoin| DescCustomerName|DescCustomerAddress|OperatorName|
+-----------+------------+---------------------------------+----------------------------+-----------------+-------------------+------------+
|          1|972541185146|                               54|                         972|     Eugene Huang|         2243 W St.|       אורנג|
|          2|972547493463|                               54|                         972|     Ruben Torres|   5844 Linden Land|       אורנג|
|          3|972549738380|                               54|                         972|      Christy Zhu|   1825 Village Pl.|       אורנג|
|          4|972540643054|                               54|                         972|Elizabeth Johnson|7553 Harness Circle|       אורנג|
|          5|

In [24]:
# Assuming df_other is the DataFrame you want to join with


customer_mrr = spark.read.parquet(r'C:\Users\alex\Desktop\TheVoice - PySpark\TheVoice - parquet_files\customer.parquet')
customer_mrr.join(customer_lines_mrr.select('PHONE_NO'), customer_mrr['CUST_NUMBER'] == customer_lines_mrr['PHONE_NO'], how='inner').show(5)


+-----------+------------+-----------------+-------------------+--------------------+-----------+------------+
|customer_id| CUST_NUMBER|        cust_name|            address|         insert_date|update_date|    PHONE_NO|
+-----------+------------+-----------------+-------------------+--------------------+-----------+------------+
|          1|972541185146|     Eugene Huang|         2243 W St.|2014-01-15 16:25:...|       NULL|972541185146|
|          2|972547493463|     Ruben Torres|   5844 Linden Land|2014-01-15 16:25:...|       NULL|972547493463|
|          3|972549738380|      Christy Zhu|   1825 Village Pl.|2014-01-15 16:25:...|       NULL|972549738380|
|          4|972540643054|Elizabeth Johnson|7553 Harness Circle|2014-01-15 16:25:...|       NULL|972540643054|
|          5|972546595053|       Julio Ruiz|7305 Humphrey Drive|2014-01-15 16:25:...|       NULL|972546595053|
+-----------+------------+-----------------+-------------------+--------------------+-----------+------------+
o

In [26]:
opfileopp_mrr.show()

+-----+----------+------+
|OPCCC|     OPDDD|prepre|
+-----+----------+------+
|   50|    פלאפון|    50|
|   52|     סלקום|    52|
|   53|הוט מובייל|    53|
|   54|     אורנג|    54|
|   58|גולן טלקום|    58|
+-----+----------+------+

